In [27]:
import os, json
import pandas as pd
import glob
from geopy.geocoders import Nominatim
import numpy as np
from tqdm import tqdm
import multiprocessing as mp
from multiprocesspandas import applyparallel
tqdm.pandas()

In [29]:
path_to_json = 'C:\\Users\\Patrick\\iCloudDrive\\Python\\bic2geo\\SwiftCodes-master\\AllCountries'
geolocator = Nominatim(user_agent='my_user_agent')

temp = pd.DataFrame()
json_pattern = os.path.join(path_to_json, '*.json')
file_list = glob.glob(json_pattern)

for file in file_list:
    data = pd.read_json(file, orient = str)
    temp = temp.append(data, ignore_index = True)

temp2 = pd.json_normalize(temp['list'])

BIC_DATA = temp.join(temp2)
del temp
del temp2
del file_list

BIC_DATA = BIC_DATA.drop('id',1)
BIC_DATA = BIC_DATA.drop('list',1)
BIC_DATA.insert(0, "id", range(1, 1+len(BIC_DATA)))
BIC_DATA['geocode'] =  BIC_DATA['city']+','+BIC_DATA['country']


In [3]:
BIC_DATA

,id,country,country_code,bank,city,branch,swift_code,geocode
0,1,Andorra,AD,ANDORRA BANC AGRICOL REIG S.A.,LES ESCALDES,None,BACAADAD,"LES ESCALDES,Andorra"
1,2,Andorra,AD,ANDORRA GESTIO AGRICOL REIG SAU,ESCALDES-ENGORDANY,None,AAMAADAD,"ESCALDES-ENGORDANY,Andorra"
2,3,Andorra,AD,BANC SABADELL D'ANDORRA S.A.,ANDORRA LA VELLA,None,BSANADAD,"ANDORRA LA VELLA,Andorra"
3,4,Andorra,AD,BANCA PRIVADA D'ANDORRA S.A.,LES ESCALDES,None,CASBADAD,"LES ESCALDES,Andorra"
4,5,Andorra,AD,"CREDIT ANDORRA,S.A.",ANDORRA LA VELLA,None,CRDAADAD,"ANDORRA LA VELLA,Andorra"
...,...,...,...,...,...,...,...,...
112882,112883,Zimbabwe,ZW,THE TETRAD GROUP,HARARE,None,TEGRZWH1,"HARARE,Zimbabwe"
112883,112884,Zimbabwe,ZW,TRUST CORPORATE SECURITIES (PRIVATE) LIMITED,HARARE,None,TCSPZWH1,"HARARE,Zimbabwe"
112884,112885,Zimbabwe,ZW,ZIMBABWE BANKING CORPORATION LTD.,BULAWAYO,None,ZBCOZWHXBYO,"BULAWAYO,Zimbabwe"
112885,112886,Zimbabwe,ZW,ZIMBABWE BANKING CORPORATION LTD.,HARARE,None,ZBCOZWHX,"HARARE,Zimbabwe"


In [84]:
LOC_DATA = pd.DataFrame(BIC_DATA['geocode'])
LOC_DATA = LOC_DATA.drop_duplicates()
LOC_DATA['loc'] = LOC_DATA['geocode'].progress_apply(geolocator.geocode, timeout=None)
LOC_DATA['point'] = LOC_DATA['loc'].progress_apply(lambda loc: tuple(loc.point)if loc else None)
LOC_DATA[['latitude', 'longitude', 'altitude']] = pd.DataFrame(LOC_DATA['point'].to_list(), index=LOC_DATA.index)

In [91]:
LOC_DATA

,geocode,loc,point,latitude,longitude,altitude
0,"LES ESCALDES,Andorra","(les Escaldes, Escaldes-Engordany, AD700, Ando...","(42.5090019, 1.5404067, 0.0)",42.509002,1.540407,0.0
1,"ESCALDES-ENGORDANY,Andorra","(Escaldes-Engordany, AD700, Andorra, (42.48691...","(42.48691275, 1.6070001140292214, 0.0)",42.486913,1.607000,0.0
2,"ANDORRA LA VELLA,Andorra","(Andorra la Vella, AD500, Andorra, (42.5069391...","(42.5069391, 1.5212467, 0.0)",42.506939,1.521247,0.0
9,"DUBAI,United Arab Emirates","(دبي, 39205, الإمارات العربية المتحدة, (25.265...","(25.2653471, 55.2924914, 0.0)",25.265347,55.292491,0.0
12,"ABU DHABI,United Arab Emirates","(نادي العاصمة للفروسية +971501222551, أبو ظبي,...","(24.4538352, 54.3774014, 0.0)",24.453835,54.377401,0.0
...,...,...,...,...,...,...
112823,"HARARE,Zimbabwe","(Harare, Harare Province, 04-263, Zimbabwe, (-...","(-17.831773, 31.045686, 0.0)",-17.831773,31.045686,0.0
112858,"BULAWAYO,Zimbabwe","(Bulawayo, Bulawayo Province, Zimbabwe, (-20.1...","(-20.1560599, 28.5887063, 0.0)",-20.156060,28.588706,0.0
112870,"CHINHOYI,Zimbabwe","(Chinhoyi, Makonde, Mashonaland West, Zimbabwe...","(-17.3615308, 30.1929353, 0.0)",-17.361531,30.192935,0.0
112871,"GWERU,Zimbabwe","(Gweru, Midlands, Zimbabwe, (-19.4616316, 29.8...","(-19.4616316, 29.8205953, 0.0)",-19.461632,29.820595,0.0


In [92]:
final = pd.merge(BIC_DATA,LOC_DATA,on='geocode',how='left')

In [94]:
final.to_csv('BIC2GEO.csv')